In [1]:
import numpy as np
import six
import sys
import os
import traceback
import re
import pickle
from copy import deepcopy

from chainer import cuda
from context2vec.common.context_models import Toks
from context2vec.common.model_reader import ModelReader
import sklearn
import pandas as pd
import logging
from scipy.stats import spearmanr
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import gensim
import math
import collections
import argparse
import h5py

from collections import defaultdict
from gensim.models import KeyedVectors
from scipy.stats import pearsonr
FLOAT = np.float32


[nltk_data] Downloading package stopwords to /home/ql261/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:

def produce_top_n_simwords(w_filter,context_embed,n_result,index2word,debug=False):
        #assume that w_filter is already normalized
        context_embed = context_embed / xp.sqrt((context_embed * context_embed).sum())
        s = xp.sqrt((w_filter * w_filter).sum(1))
        s[s==0.] = 1.
        w_filter /= s.reshape((s.shape[0], 1))
        similarity_scores=[]
#         print('producing top {0} simwords'.format(n_result))
        similarity = (w_filter.dot(context_embed)+1.0)/2
        top_words_i=[]
        top_words=[]
        count = 0
        for i in (-similarity).argsort():
                    if xp.isnan(similarity[i]):
                        continue
                    if debug==True:
                        try:
                            print('{0}: {1}'.format(str(index2word[int(i)]), str(similarity[int(i)])))
                        except UnicodeEncodeError as e:
                            print (e)
                            
                    count += 1
                    top_words_i.append(int(i))
                    top_words.append(index2word[int(i)])
                    similarity_scores.append(float(similarity[int(i)]))
                    if count == n_result:
                        break

        top_vec=w_filter[top_words_i,:]
        return top_vec,xp.array(similarity_scores),top_words

def top_cluster_density(top_vec,similarity_scores):
    #normalize the top_vec
    s = xp.sqrt((top_vec * top_vec).sum(1))
    s[s==0.] = 1.
    top_vec = top_vec/ s.reshape((s.shape[0], 1))
    #perform the centroid
    max_score=similarity_scores[0]
    similarity_scores=xp.array(similarity_scores).reshape(len(similarity_scores),1)/sum(similarity_scores)
    centroid_vector=sum(top_vec*similarity_scores)
    # average of cosine distance to the centroid,weighted by max scores
    inf_score=float(sum(top_vec.dot(centroid_vector))/len(top_vec)*max_score)
    return inf_score

In [3]:
def load_w2salience(model_w2v,w2salience_f,weight_type):
    w2salience={}
    with open(w2salience_f) as f:
        for line in f:
            line=line.strip()
            if line=='':
                continue
            if line.startswith('sentence total'):
                sent_total=int(line.split(':')[1])
                continue
            w,w_count,s_count=line.split('\t')
            if model_w2v.wv.__contains__(w):
                if weight_type==INVERSE_W_FREQ:
                    w2salience[w]=1/float(w_count)
                elif weight_type==INVERSE_S_FREQ:
                    w2salience[w]=math.log(1+sent_total/float(s_count))
    #                 w2salience[w]=math.log(1+84755431/float(s_count))
    return w2salience

def skipgram_context(model,words,pos,weight=None,w2entropy=None):
    context_wvs=[]
    weights=[]
    for i,word in enumerate(words):
        if i != pos: #surroudn context words
            if word in model:
                if weight ==LDA:
                    if word in w2entropy:
                        weights.append(1/(w2entropy[word]+1.0))
                        context_wvs.append(model[word])
        
                elif weight in [INVERSE_W_FREQ,INVERSE_S_FREQ]:
                    if word in w2entropy:
                        model[word]
                        weights.append(w2entropy[word])
                        context_wvs.append(model[word])
                    
                else:
                    #equal weights per word
                    context_wvs.append(model[word])
                    weights.append(1.0)
            else:
                pass
#                 print ('==warning==: key error in context {0}'.format(word))
#     print ('per word weights',weights)
    context_embed=sum(np.array(context_wvs)*np.array(weights).reshape(len(weights),1))#/sum(weights)
#     print ('skipgram context sum:', context_embed[:10])
    return sum(weights),context_embed #  will be normalized later

def lg_model_out_w2v(top_words,w_target,word2index_target):
        # lg model substitutes in skipgram embedding
        top_vec=[]
        index_list=[]
        for i,word in enumerate(top_words):
            try :
                top_vec.append(w_target[word2index_target[word]])
                index_list.append(i)
            except KeyError as e:
                pass
#                 print ('lg subs {0} not in w2v'.format(e))
        if top_vec==[]:
            print ('no lg subs in w2v space')
            return xp.array([]),[]
        else:
            return xp.stack(top_vec),index_list
    
def context_inform(test_s,test_w, model,model_type,n_result,w_filter,index2word,weight,w2entropy=None,w_target=None,word2index_target=None,index2word_target=None):
    #produce context representation and infromative score for each context

    test_s=test_s.replace(test_w, ' '+test_w+' ')
    words=test_s.split()
    pos=words.index(test_w)

    score=1.0 #default score
    
    # Decide on the model
    if model_type=='context2vec':
            context_embed= model.context2vec(words, pos)

    elif model_type=='skipgram' or model_type=='a la carte':
        score,context_embed=skipgram_context(model,words,pos,weight,w2entropy)
        context_embed_out=xp.array(context_embed)
        
    elif model_type=='context2vec-skipgram':
        context_embed= model.context2vec(words, pos)

        top_vec,sim_scores,top_words=produce_top_n_simwords(w_filter,context_embed,n_result,index2word,debug=True)
        top_vec,index_list=lg_model_out_w2v(top_words,w_target,word2index_target) 
        sim_scores=sim_scores[index_list] #weighted by substitute probability
        if weight==SUBSTITUTE_PROB:
            context_embed_out=xp.array(sum(top_vec*sim_scores.reshape(len(sim_scores),1)))
        else:
            context_embed_out=xp.array(sum(top_vec*((sim_scores/sum(sim_scores)).reshape(len(sim_scores),1))))
        
    else:
        print ('model type {0} not recognized'.format(model_type))
        sys.exit(1)
        
    #decide on weight per sentence
    
    if weight==TOP_MUTUAL_SIM:
#         if word2index_target==None: #not context2vec-skipgram
#             context2vec word embedding space neighbours
        top_vec,sim_scores,top_words=produce_top_n_simwords(w_filter,context_embed,n_result,index2word)
        #skipgram word embedding space neighbours when context2vec-skipgram
        score=top_mutual_sim(top_vec,sim_scores)
    elif weight==TOP_CLUSTER_DENSITY:

        top_vec,sim_scores,top_words=produce_top_n_simwords(w_filter,context_embed,n_result,index2word)
        score=top_cluster_density(top_vec,sim_scores)
    elif weight==SUBSTITUTE_PROB:
        score=sum(sim_scores)
        print ('substitute prob score',score)
    elif weight=='learned':
        print ('learned not implemented')
    elif weight=='gaussian':
        print ('gaussian not implemented')
    elif weight ==False or weight in [LDA,INVERSE_S_FREQ,INVERSE_W_FREQ]:
        score=score
    else:
        print ('weight mode {0} not recognized'.format(weight))
    return float(score),context_embed_out

def additive_model(test_ss,test_w, model_type,model,n_result,w_filter,index2word,weight=False,w2entropy=None,w_target=None,word2index_target=None,index2word_target=None,f_w=None,context2vec_preembeds=None,scores=None,M=None):
    #produce context representation across contexts using weighted average
    print ('model type is :{0}'.format(model_type))
    context_out=[]
    context_weights=[]
    for test_id in range(len(test_ss)):
        test_s=test_ss[test_id]
        test_s=test_s.lower().strip()
        #produce context representation with scores
        if type(context2vec_preembeds)!=type(None):
            context_embed=xp.array(context2vec_preembeds[test_id])
            score=float(scores[test_id])
        else:
            
            score,context_embed=context_inform(test_s,test_w, model,model_type,n_result,w_filter,index2word,weight,w2entropy,w_target,word2index_target,index2word_target)
        
        if score==0 or context_embed.all()==0:
            print ('empty context vector')
           
        else:
            context_out.append(context_embed)
            context_weights.append(score)
#     print ('context_weights',context_weights)
    #sum representation across contexts
    if context_out==[]:
        return None
    else:
        context_out=xp.stack(context_out)
    
    print (type(M))
    if type(M)!=type(None):
        context_avg=sum(context_out)
        context_avg=M.dot(context_avg)
        print ('add alacarte')
        #.dot(M.T)

    elif model_type=='skipgram' or weight==SUBSTITUTE_PROB:
        # context representation by weighted sum of all context words in all contexts
#         print ('context out: ', context_out[:10])
#         print ('context_weights',context_weights)
        context_avg=sum(context_out)/sum(context_weights)
   
    else:
        norm_weights=xp.array(context_weights).reshape(len(context_weights),1)/float(sum(context_weights))
        if f_w!=None:
            f_w.write(','.join([str(i[0]) for i in norm_weights])+'\n')
        # context represenatation by weighted sum of contexts
        
        context_avg=sum(norm_weights*context_out)
    
    
    # check new embedding neighbours

    print('producing top {0} words for new embedding'.format(n_result))
    if index2word_target==None:
        top_vec,scores,top_words=produce_top_n_simwords(w_filter,context_avg,n_result,index2word,debug=True)
    else:
        #print the target space neighbours for context2vec-skipgram
        top_vec,scores,top_words=produce_top_n_simwords(w_target,context_avg,n_result,index2word_target,debug=True)
    return context_avg


def contexts_per_tgw(sents,model_type,context_model,n_result,w,index2word,weight,w2entropy,w_target,word2index_target,index2word_target,context2vec_preembeds=None,scores=None,M=None):
    
    if model_type=='context2vec-skipgram?skipgram':
#             
            #context2vevc      
            context_avg_1=additive_model(sents,'___', model_type.split('?')[0],context_model[0],n_result,w[0],index2word[0],weight[0],w2entropy[0],w_target[0],word2index_target[0],index2word_target[0],context2vec_preembeds=context2vec_preembeds,scores=scores,M=M)
            #skipgram

            context_avg_2=additive_model(sents,'___', model_type.split('?')[1],context_model[1],n_result,w[1],index2word[1],weight[1],w2entropy[1],w_target[1],word2index_target[1],index2word_target[1],M=M)

            if type(context_avg_1)!=type(None) and type(context_avg_2)!=type(None):
                context_avg=(context_avg_1+context_avg_2)/2

            elif type(context_avg_1)!=type(None):
                context_avg=context_avg_1

            elif type(context_avg_2)!=type(None):
                context_avg=context_avg_2
            else:
                context_avg=None
            
    else:

            context_avg=additive_model(sents,'___', model_type,context_model,n_result,w,index2word,weight[0],w2entropy,w_target,word2index_target,index2word_target,context2vec_preembeds=context2vec_preembeds,scores=scores,M=M)
#             print ('context avg out', context_avg[:10])
    return context_avg
  
def output_embedding(w,w_target,word2index,word2index_target):
    if model_type=='context2vec-skipgram?skipgram':
        #compute probe embeddings in skipgram space
            w_out=w[1]
            w_target_out=w_target[1]
            word2index_out=word2index[1]
            word2index_target_out=word2index_target[1]
    else:
            w_out=w
            w_target_out=w_target
            word2index_out=word2index
            word2index_target_out=word2index_target
    if word2index_target_out==None:
        return w_out,word2index_out
    else:
        return w_target_out,word2index_target_out

In [4]:
def filter_w(w,word2index,index2word):
    #filter out words with no letters in, and stopwords
    stopw=stopwords.words('english')
    stopw=[word.encode('utf-8') for word in stopw]
    index2word_filter={}
    word2index_filter={}
    index_filter2index=[]
    counter=0
    for word in word2index:
            if word not in stopw: #and re.search(r'[^a-zA-Z]',word)==None:
                    index_filter2index.append(word2index[word])
                    word2index_filter[word]=counter
                    index2word_filter[counter]=word
                    counter+=1
    w_filter= w[index_filter2index,:]
    return w_filter,word2index_filter,index2word_filter

def rm_stopw_context(model):
    stopw=stopwords.words('english')
    stopw=[word.encode('utf-8') for word in stopw]
    
    model2={word:model.wv.__getitem__(word) for word in model.wv.vocab if word not in stopw}
    return model2




In [5]:
def cosine(context_avg,probe_w_vec):
    if context_avg.all()==0. or probe_w_vec.all()==0.:
        return 0.
        
    context_avg = context_avg / xp.sqrt((context_avg * context_avg).sum())
    probe_w_vec=probe_w_vec/xp.sqrt((probe_w_vec*probe_w_vec).sum())
    cos=float(probe_w_vec.dot(context_avg))
    print (cos)
    if np.isnan(cos):
        print ('Warning: cos is nan')
        sys.exit(1)
    return cos

def preprocess_nonce(sent,contexts):
    
    sents_out=[]
    
    sent=sent.lower()
    results=re.finditer('___ ',sent)
    matches=[m for m in results]
    for i in range(len(matches)):
        sent_masked=sent
        matches_mask=[(m2.start(0),m2.end(0)) for i2,m2 in enumerate(matches) if i2!=i]
        matches_mask=sorted(matches_mask, key=lambda x:x[0],reverse=True)
        for m in matches_mask:
            sent_masked=sent_masked[:m[0]]+sent_masked[m[1]:]
        sents_out.append(sent_masked+' .')
    return sents_out

def update_mrr(nns,nonce,mrr,ranks):
    rr = 0
    n = 1
    for nn in nns:
        word = nn[0]
        if word == nonce:
            print (word)
            rr = n
            ranks.append(rr)
        else:
            n+=1

    if rr != 0:
        mrr+=float(1)/float(rr)	
    print rr,mrr
    return mrr,ranks

def eval_nonce(nonce_data_f,context_model,model_w2v,model_type,n_result,w,index2word,word2index,weight=False,w2entropy=None,w_target=None,word2index_target=None,index2word_target=None,contexts=None,M=None):
        #read in contexts
        ranks = []
        mrr = 0.0
        data=pd.read_csv(os.path.join(nonce_data_f),delimiter='\t',header=None,comment='#')
        
        for index, row in data.iterrows():
            if index>100 and index%100==0:
                print (index)
            sents=preprocess_nonce(row[1],contexts)
            nonce=row[0]
            if nonce not in model_w2v:
                print ('{0} not known'.format(nonce))
                continue
            context_avg=contexts_per_tgw(sents,model_type,context_model,n_result,w,index2word,weight,w2entropy,w_target,word2index_target,index2word_target,M=M)
            if xp==cuda.cupy:
                context_avg=xp.asnumpy(context_avg)
                
            # MRR Rank calculation
            nns=model_w2v.similar_by_vector(context_avg,topn=len(model_w2v.wv.vocab))
            mrr,ranks=update_mrr(nns,nonce,mrr,ranks)

        print ("Final MRR: ",mrr,len(ranks),float(mrr)/float(len(ranks)))
        print ('mean: ', np.mean(ranks))
        print ('mediam : {0}'.format(np.median(ranks)))
        return ranks
def load_transform(Afile,model):
    '''loads the transform from a text file
    Args:
    Afile: string; transform file name
    Returns:
    numpy array
    '''
    if Afile.endswith('bin'):
        M = np.fromfile(Afile, dtype=FLOAT)
        d = int(np.sqrt(M.shape[0]))
        print (d)
        assert d == next(iter(model.values())).shape[0], "induction matrix dimension and word embedding dimension must be the same"
        M = M.reshape(d, d)
        M=xp.array(M)
        return M
    elif Afile.endswith('txt'):
        with open(Afile, 'r') as f:
            return xp.array(np.vstack([np.array([FLOAT(x) for x in line.split()]) for line in f]))
            
def eval_nonce_withcontexts(nonce_data_f,context_model,model_w2v,model_type,n_result,w,index2word,word2index,weight=False,w2entropy=None,w_target=None,word2index_target=None,index2word_target=None,trials=100,M=None):

    data=pd.read_csv(os.path.join(nonce_data_f),delimiter='\t',header=None,comment='#')
    ranks=defaultdict(lambda: defaultdict(list))
    mrrs=defaultdict(lambda: defaultdict(int))
    context2vec_preembeds_all=None
    context2vec_preembeds=None
    scores_all=None
    orders_inf=None
    scores=None
#     start evaluation
    contexts_f=os.path.join(os.path.dirname(nonce_data_f),'contexts')
    
    for index, row in data.iterrows():
        rw=row[0]
        if rw not in model_w2v:
                print ('{0} not known'.format(rw))
                continue
        contexts_w_f=os.path.join(contexts_f,rw+'.txt')
        if not os.path.isfile(contexts_w_f):
            print ('{0} does not have contexts'.format(rw))
            continue
            
        print ('\n==========processing rareword {0}'.format(rw))

        #contexts:
        #load sentences
        sents_all=load_sents(contexts_w_f,rw)
        #load contexts
        if 'context2vec' in model_type.split('?')[0]:
                if model_type=='context2vec-skipgram?skipgram':
                    context2vec_preembeds_all,scores_all=load_contexts(rw,sents_all,context_model[0],model_type.split('?')[0],n_result,w[0],index2word[0],weight[0],w2entropy[0],w_target[0],word2index_target[0],index2word_target[0])
                elif model_type=='context2vec-skipgram':
                    context2vec_preembeds_all,scores_all=load_contexts(rw,sents_all,context_model,model_type.split('?')[0],n_result,w,index2word,weight[0],w2entropy,w_target,word2index_target,index2word_target)
                orders_inf=(-scores_all).argsort()
        #do trials
        for trial in range(trials):
            print ('\n=====Trial no. {0}'.format(trial))
            perm = np.random.permutation(255)
            for logfreq in range(8):
                freq = 2**logfreq
                print ('\ncontext num is {0}'.format(freq))
                context2vec_preembeds, scores,sents=produce_contexts_per_trial(trials,freq,perm,scores_all,context2vec_preembeds_all,orders_inf,sents_all)
                context_avg=contexts_per_tgw(sents,model_type,context_model,n_result,w,index2word,weight,w2entropy,w_target,word2index_target,index2word_target,context2vec_preembeds,scores,M=M)
                
                if type(context_avg)!=type(None):
                    if xp==cuda.cupy:
                        context_avg=xp.asnumpy(context_avg)
                
                    # MRR Rank calculation
                    nns=model_w2v.similar_by_vector(context_avg,topn=len(model_w2v.wv.vocab))
                    mrrs[trial][freq],ranks[trial][freq]=update_mrr(nns,rw,mrrs[trial][freq],ranks[trial][freq])
    mrr_res=defaultdict(list)
    median_res=defaultdict(list)
    for trial in mrrs:
        for freq in mrrs[trial]:
            mrr_res[freq].append(float(mrrs[trial][freq])/float(len(ranks[trial][freq])))
            median_res[freq].append(np.median(ranks[trial][freq]))
    print ('{0}\t{1}\t{2}\t{3}\t{4}'.format('freq','MRR','MRR_STD','MEDIAN','MEDIAN_STD'))
    for freq in sorted(mrr_res.keys()):
        print ('{0}\t{1}\t{2}\t{3}\t{4} '.format(freq,np.mean(np.array(mrr_res[freq])),np.std(np.array(mrr_res[freq])),np.mean(np.array(median_res[freq])),np.std(np.array(median_res[freq]))))
    return mrr_res,median_res

def eval_chimera(chimeras_data_f,context_model,model_type,n_result,w,index2word,word2index,weight=False,w2entropy=None,w_target=None,word2index_target=None,index2word_target=None,M=None):
    chimeras_data_dir=os.path.dirname(chimeras_data_f)
    num_sent=os.path.basename(chimeras_data_f).split('.')[1][1]
    print (chimeras_data_dir)
    print (num_sent)
    with open(chimeras_data_dir+'/weights_{0}_{1}_{2}'.format(num_sent,model_type,str(weight)),'w') as f_w:
        spearmans=[]
        data=pd.read_csv(os.path.join(chimeras_data_f),delimiter='\t',header=None)
        w_target_out,word2index_target_out=output_embedding(w,w_target,word2index,word2index_target)
        for index, row in data.iterrows():
            if index>100 and index%100==0:
                print (index)
            golds=[]
            model_predict=[]
            probes=[]
            sents=row[1].lower().split('@@')
            #compute context representation
            context_avg=contexts_per_tgw(sents,model_type,context_model,n_result,w,index2word,weight,w2entropy,w_target,word2index_target,index2word_target,M=M)
            #cosine similarity with probe embedding
            for gold,probe in zip(row[3].split(','),row[2].split(',')):
                try:
                    probe_w_vec=w_target_out[word2index_target_out[probe]]
                    cos=cosine(context_avg,probe_w_vec)
                    model_predict.append(cos)
                    golds.append(gold)
                    probes.append(probe)
                except KeyError as e:
                    print ("====warning key error for probe=====: {0}".format(e))
            print ('probes',probes)
            print ('gold',golds)
            print ('model_predict',model_predict)
            sp=spearmanr(golds,model_predict)[0]
            print ('spearman correlation is {0}'.format(sp))
            if not math.isnan(sp):
                spearmans.append(sp)
        print ("AVERAGE RHO:",float(sum(spearmans))/float(len(spearmans)))


def load_contexts(rw,sents,model,model_type,n_result,w_filter,index2word,weight,w2entropy,w_target,word2index_target,index2word_target):          
    print ('loading contexts..')
    contexts=[]
    scores=[]
    for i,test_s in enumerate(sents):
        test_s=sents[i]
        test_s=test_s.replace(TARGET_W, ' '+TARGET_W+' ')
        print(i),
        words=test_s.split()
        pos=words.index(TARGET_W)
        score,context_embed=context_inform(test_s,TARGET_W, model,model_type,n_result,w_filter,index2word,weight,w2entropy,w_target,word2index_target,index2word_target)
        if context_embed.all()==0:
            context_embed=xp.zeros(w_target.shape[-1])
            score=0
        contexts.append(context_embed)
        scores.append(score)
    contexts=xp.stack(contexts,axis=0)
    scores=np.array(scores)
    print ('\ncontexts for {0} completed'.format(rw))
    return contexts,scores

def load_sents(contexts_w_f,rw):
    sents_all=[]
    with open (contexts_w_f) as f:
        for line in f:
            line=line.replace(rw,TARGET_W).replace(rw.replace('_',' '),TARGET_W).replace(rw.replace('-',' '),TARGET_W).strip().lower()
            if not line.endswith('.'):
                line=line+' .'
            sents_all.append(line)
    sents_all=np.array(sents_all)
    return sents_all

def produce_contexts_per_trial(trials,freq,perm,scores_all,context2vec_preembeds_all,orders_inf,sents_all):
    if trials==1 and type(scores_all)!=type(None):
        context_inds=orders_inf[:freq]
    else:
        context_inds=perm[np.array(range(freq-1, 2*freq-1)),]

    sents=sents_all[np.array(context_inds),]
    print (context_inds)
    if type(context2vec_preembeds_all)!=type(None):
        context2vec_preembeds=context2vec_preembeds_all[context_inds,]
        scores=scores_all[context_inds,]
        return context2vec_preembeds, scores,sents
    else:
        return None,None,sents
    

def eval_crw_stf(crw_stf_f,model_param_f,context_model,model_type,n_result,w,index2word,word2index,weight=False,w2entropy=None,w_target=None,word2index_target=None,index2word_target=None,trials=100,M=None):
    data=pd.read_csv(os.path.join(crw_stf_f),delimiter='\t',header=None,comment='#')
    model_predicts=defaultdict(lambda: defaultdict(list))
    golds=defaultdict(lambda: defaultdict(list))
    context2vec_preembeds_all=None
    context2vec_preembeds=None
    scores_all=None
    orders_inf=None
    scores=None
    rw_prev=''
#     start evaluation
    w_target_out,word2index_target_out=output_embedding(w,w_target,word2index,word2index_target)
    contexts_f=os.path.join(os.path.dirname(crw_stf_f),'context')
    
    for index, row in data.iterrows():
        probe_w=row[0]
        rw=row[1]
        if probe_w not in word2index_target_out: 
            continue
        gold=row[2]
        print ('\n==========processing rareword {0}'.format(rw))

        #load sentences
        contexts_w_f=os.path.join(contexts_f,rw+'.txt')
        sents_all=load_sents(contexts_w_f,rw)
        #load contexts
        if 'context2vec' in model_type.split('?')[0]:
            if rw_prev!=rw:
                rw_prev=rw
                if model_type=='context2vec-skipgram?skipgram':
                    context2vec_preembeds_all,scores_all=load_contexts(rw,sents_all,context_model[0],model_type.split('?')[0],n_result,w[0],index2word[0],weight[0],w2entropy[0],w_target[0],word2index_target[0],index2word_target[0])
                elif model_type=='context2vec-skipgram':
                    context2vec_preembeds_all,scores_all=load_contexts(rw,sents_all,context_model,model_type.split('?')[0],n_result,w,index2word,weight[0],w2entropy,w_target,word2index_target,index2word_target)
                orders_inf=(-scores_all).argsort()
        #do trials
        for trial in range(trials):
            print ('\n=====Trial no. {0}'.format(trial))
            perm = np.random.permutation(255)
            for logfreq in range(8):
                freq = 2**logfreq
                print ('\ncontext num is {0}'.format(freq))
                context2vec_preembeds, scores,sents=produce_contexts_per_trial(trials,freq,perm,scores_all,context2vec_preembeds_all,orders_inf,sents_all)
                context_avg=contexts_per_tgw(sents,model_type,context_model,n_result,w,index2word,weight,w2entropy,w_target,word2index_target,index2word_target,context2vec_preembeds,scores,M=M)
                
                if type(context_avg)!=type(None):
                    #cosine similarity
                    probe_w_vec=w_target_out[word2index_target_out[probe_w]]
                    cos=cosine(context_avg,probe_w_vec)
                    model_predicts[trial][freq].append(cos)
                    golds[trial][freq].append(gold)
     
    #
    sps=defaultdict(list)
    for trial in model_predicts:
        for freq in model_predicts[trial]:
            sp=spearmanr(golds[trial][freq],model_predicts[trial][freq])[0]
            sps[freq].append(sp)
    print ('{0}\t{1}\t{2}'.format('freq','SPEARMAN RANKS MEAN','SPEARMAN RANKS STD'))
    for freq in sorted(sps.keys()):
        print ('{0}\t{1}\t{2} '.format(freq,np.mean(np.array(sps[freq])),np.std(np.array(sps[freq]))))
    return model_predicts,sps

            
            
            
def eval_card(card_f,model_param_f,context_model,model_type,n_result,w,index2word,word2index,weight=False,w2entropy=None,w_target=None,word2index_target=None,index2word_target=None,trials=100,contexts=False):
    data=pd.read_csv(os.path.join(card_f),delimiter='\t',header=None)
    model_predicts=defaultdict(lambda: defaultdict(list))
    golds=defaultdict(lambda: defaultdict(list))
    golds_exist=[]
    model_preds_exist=[]
    context2vec_preembeds_all=None
    context2vec_preembeds=None
    scores_all=None
    orders_inf=None
    scores=None
    
    missing_w=defaultdict(list)
    w_target_out,word2index_target_out=output_embedding(w,w_target,word2index,word2index_target)
    contexts_f=os.path.join(os.path.dirname(card_f),'contexts')
    
    for index, row in data.iterrows():
        print (index)
        rw_1=str(row[0])
        rw_2=str(row[1])
        gold=float(row[2])
        rw_embeds=[]
        print (rw_1,rw_2,gold)
        
        #give unseen words 0 embedding
        for rw in [rw_1,rw_2]:
            if rw not in word2index_target_out:
                if contexts==True:
                    contexts_w_f=os.path.join(contexts_f,rw.lower().replace('/','_')+'.txt')
                    if os.path.isfile(contexts_w_f):
                            rw_embeds.append((rw,contexts_w_f))
                            continue
                rw_embeds.append(np.zeros(len(w_target_out[0])))
                missing_w[index].append(rw)
            else:
                rw_embeds.append(w_target_out[word2index_target_out[rw]])
                
        # load contexts
        if  not any(type(rw_embed)==tuple for rw_embed in rw_embeds):
            model_preds_exist.append(cosine(rw_embeds[1],rw_embeds[0]))
            golds_exist.append(gold)
        elif any(type(rw_embed)!=tuple and rw_embed.all()==0 for rw_embed in rw_embeds):
            model_preds_exist.append(0.)
            golds_exist.append(gold)
        else:
#         elif (type(rw_embeds[0])==tuple and not rw_embeds[1].all()==0) or (type(rw_embeds[1])==tuple and not rw_embeds[0].all()==0):
            for i,rw_embed in enumerate(rw_embeds):
                if type(rw_embed)==tuple:
                    #load sentences
                    rw,contexts_w_f=rw_embed
                    sents_all=load_sents(contexts_w_f,rw.lower())
#                         load contexts
                    if 'context2vec' in model_type.split('?')[0]:
                            if model_type=='context2vec-skipgram?skipgram':
                                context2vec_preembeds_all,scores_all=load_contexts(rw,sents_all,context_model[0],model_type.split('?')[0],n_result,w[0],index2word[0],weight[0],w2entropy[0],w_target[0],word2index_target[0],index2word_target[0])
                            elif model_type=='context2vec-skipgram':
                                context2vec_preembeds_all,scores_all=load_contexts(rw,sents_all,context_model,model_type.split('?')[0],n_result,w,index2word,weight[0],w2entropy,w_target,word2index_target,index2word_target)
                            orders_inf=(-scores_all).argsort()
                    rw_embeds[i]=(context2vec_preembeds_all,scores_all,orders_inf,sents_all)
            #do trials
            for trial in range(trials):
                print ('\n=====Trial no. {0}'.format(trial))
                perm = np.random.permutation(255)
                for logfreq in range(8):
                    freq = 2**logfreq
                    print ('\ncontext num is {0}'.format(freq))
                    rw_out=[]
                    for i, rw_embed in enumerate(rw_embeds):
                        if type(rw_embed)==tuple:
                            context2vec_preembeds_all,scores_all,orders_inf,sents_all=rw_embed
                            context2vec_preembeds, scores,sents=produce_contexts_per_trial(trials,freq,perm,scores_all,context2vec_preembeds_all,orders_inf,sents_all)
                            context_avg=contexts_per_tgw(sents,model_type,context_model,n_result,w,index2word,weight,w2entropy,w_target,word2index_target,index2word_target,context2vec_preembeds,scores)
                            rw_out.append(context_avg)
                        else:
                            rw_out.append(rw_embed)
                    if all (type(rw_o)!=type(None) for rw_o in rw_out):
                        #cosine similarity
                        cos=cosine(rw_out[0],rw_out[1])
                        model_predicts[trial][freq].append(cos)
                        golds[trial][freq].append(gold)
                        print (model_predicts[trial][freq])
                        print (golds[trial][freq])



    sps=defaultdict(list)
    prs=defaultdict(list)
    for trial in model_predicts:
        for freq in model_predicts[trial]:
            sp=spearmanr(golds_exist+golds[trial][freq],model_preds_exist+model_predicts[trial][freq])[0]
#             sp=spearmanr(golds[trial][freq],model_predicts[trial][freq])[0]

            sps[freq].append(sp)
            pr=pearsonr(golds_exist+golds[trial][freq],model_preds_exist+model_predicts[trial][freq])[0]
#             pr=pearsonr(golds[trial][freq],model_predicts[trial][freq])[0]
            prs[freq].append(pr)
    print ('{0}\t{1}\t{2}\t{3}\t{4}'.format('freq','SPEARMAN RANKS MEAN','SPEARMAN RANKS STD','PEARSON RANKS MEAN','PEARSON RANKS STD'))
    if sps.keys()!=[]:
        for freq in sorted(sps.keys()):
            print ('{0}\t{1}\t{2}\t{3}\t{4} '.format(freq,np.mean(np.array(sps[freq])),np.std(np.array(sps[freq])),np.mean(np.array(prs[freq])),np.std(np.array(prs[freq]))))
    else:
            print ('{0}\t{1}\t{2}\t{3}\t{4} '.format(None,spearmanr(golds_exist,model_preds_exist)[0],None,pearsonr(golds_exist,model_preds_exist)[0],None))
    missed_pairs=[missing_w[index] for index in missing_w]
    missed_words=[w  for index in missing_w for w in missing_w[index]]
    print ('missed pairs {0}, missed words {1}'.format(len(missed_pairs),len(missed_words)))
    


In [6]:
if __name__=="__main__":
    
    TOP_MUTUAL_SIM='top_mutual_sim'
    TOP_CLUSTER_DENSITY='top_cluster_density'
    LDA='lda'
    TARGET_W='___'
    INVERSE_S_FREQ='inverse_s_freq'
    INVERSE_W_FREQ='inverse_w_q'
    SUBSTITUTE_PROB='substitute_prob'
    WEIGHT_DICT={0:False,1:TOP_MUTUAL_SIM,2:LDA,3:INVERSE_S_FREQ,4:INVERSE_W_FREQ,5:TOP_CLUSTER_DENSITY, 6:SUBSTITUTE_PROB}

    ##### 1. params read in
    if sys.argv[0]=='/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py':
        
        ###data:
#         data='./eval_data/data-chimeras/dataset_alacarte.l2.fixed.test.txt.punct'
#         data='./eval_data/data-nonces/n2v.definitional.dataset.test.txt'
#         data='./eval_data/card-660/dataset.tsv'
        data='./eval_data/CRW/CRW-562.txt'
#         weights=[WEIGHT_DICT[0],WEIGHT_DICT[3]]
        weights=[WEIGHT_DICT[0]]
        gpu=-1
        model_type='context2vec-skipgram'
        w2salience_f=None
        matrix_f=None
        n_result=20
        trials=100
        context_flag=True
#         matrix_f='../models/ALaCarte/transform/nonce_samecorpus.bin'
        skipgram_model_f='./eval_data/CRW/vectors.txt'
#         skipgram_model_f='../models/wiki_all.model/wiki_all.sent.split.model'
#         skipgram_model_f='../models/glove/glove.840B.300d.w2vformat.txt'
#         skipgram_model_f='../models/conceptnet/numberbatch-en-17.06.txt'
#         context2vec_model_f='../models/context2vec/model_dir/MODEL-wiki.params.12'
        context2vec_model_f='../models/context2vec/model_dir/MODEL-WWCsub-400dim.params.8'
        ######w2salience_f
#         w2salience_f='../corpora/corpora/wiki.all.utf8.sent.split.tokenized.vocab'
#         w2salience_f='../corpora/corpora/WWC_norarew.txt.tokenized.vocab'
#         w2salience_f='../models/lda/w2entropy'


        if model_type=='skipgram' or model_type =='a la carte':
            model_param_file=skipgram_model_f
        elif model_type=='context2vec-skipgram':
            model_param_file=context2vec_model_f+'?'+skipgram_model_f
        elif model_type=='context2vec-skipgram?skipgram':
            model_param_file=context2vec_model_f+'?'+skipgram_model_f

    
    else:
        

        parser = argparse.ArgumentParser(description='Evauate on rare words.')
        parser.add_argument('--f',  type=str,
                            help='model_param_file',dest='model_param_file')
        parser.add_argument('--m', dest='model_type', type=str,
                            help='<model_type: context2vec; context2vec-skipgram (context2vec substitutes in skipgram space); context2vec-skipgram?skipgram (context2vec substitutes in skipgram space plus skipgram context words)>')
        parser.add_argument('--w', dest='weights', type=int, nargs='+',help='<weight:{0}>'.format (sys.argv[0],WEIGHT_DICT.items()))       
        parser.add_argument('--d', dest='data', type=str, help='data file')
        parser.add_argument('--g', dest='gpu',type=int, default=-1,help='gpu, default is -1')
        parser.add_argument('--ws', dest='w2salience_f',type=str, default=None,help='word2salience file, optional')
        parser.add_argument('--n_result',default=20,dest='n_result',type=int,help='top n result for language model substitutes')
        parser.add_argument('--t', dest='trials',type=int, default=100, help='trial number. When trial==1, only test on the most infortive contexts')
        parser.add_argument('--c', dest='context_flag',type=bool, default=False, help='context flag on the nonce experiment')
        parser.add_argument('--ma', dest='matrix_f',type=str,default=None,help='matrix file for a la carte')
        args = parser.parse_args()
        model_param_file = args.model_param_file
        model_type=args.model_type
        n_result=args.n_result
        weights=[WEIGHT_DICT[w_i] for w_i in args.weights]
        trials=args.trials 
        data =args.data
        gpu=args.gpu
        w2salience_f=args.w2salience_f
        context_flag=args.context_flag
        matrix_f=args.matrix_f
    
    #### 2. gpu setup 
   
    if gpu >= 0:
        cuda.check_cuda_available()
        cuda.get_device(gpu).use()    
    xp = cuda.cupy if gpu >= 0 else np
    
    
    #### 3. initialize according to model types
    
    def read_w2v(model_param_file):
        if not model_param_file.endswith('txt'):
            model_w2v = gensim.models.Word2Vec.load(model_param_file)
        else:
            model_w2v = KeyedVectors.load_word2vec_format(model_param_file)
       
        return model_w2v
    
    def read_context2vec(model_param_file,gpu):
        model_reader = ModelReader(model_param_file,gpu)
        w = xp.array(model_reader.w)
        index2word = model_reader.index2word
        word2index=model_reader.word2index
        model = model_reader.model
        return w,index2word,word2index,model
    
    print ('read model....')
    if model_type=='context2vec':
        #read in model
        w,index2word,word2index,model=read_context2vec(model_param_file,gpu)
        w_target=None
        word2index_target=None
        index2word_target=None
        
    elif model_type=='skipgram':
        model_w2v=read_w2v(model_param_file)
        w=xp.array(deepcopy(model_w2v.wv.vectors))
        #vector normalize for target w embedding, consistent with context2vec w and convenient for cosine computation among substitutes
        s = xp.sqrt((w * w).sum(1))
        s[s==0.] = 1.
        w /= s.reshape((s.shape[0], 1))
        
        index2word=model_w2v.wv.index2word
        word2index={key: model_w2v.wv.vocab[key].index for key in model_w2v.wv.vocab}
        w_target=None
        word2index_target=None
        index2word_target=None
        print ('filter words for context....')
        model=rm_stopw_context(model_w2v)
        
    elif model_type=='context2vec-skipgram':
        model_param_context,model_param_w2v=model_param_file.split('?')
        w,index2word,word2index,model=read_context2vec(model_param_context,gpu)
        model_w2v=read_w2v(model_param_w2v)
        w_target=xp.array(model_w2v.wv.vectors)
        index2word_target=model_w2v.wv.index2word
        word2index_target={key: model_w2v.wv.vocab[key].index for key in model_w2v.wv.vocab}
    
    elif model_type=='context2vec-skipgram?skipgram':
        model_param_context,model_param_w2v=model_param_file.split('?')
        #context2vec-skipgram
        w,index2word,word2index,model=read_context2vec(model_param_context,gpu)
        model_w2v=read_w2v(model_param_w2v)
        w_target=xp.array(model_w2v.wv.vectors)
        index2word_target=model_w2v.wv.index2word
        word2index_target={key: model_w2v.wv.vocab[key].index for key in model_w2v.wv.vocab}
    
        # skigpram
        model_skipgram = model_w2v
        w_skipgram=xp.array(deepcopy(model_skipgram.wv.vectors))
        #vector normalize for probe w embedding for calculating top vector similarity
        s = xp.sqrt((w_skipgram * w_skipgram).sum(1))
        s[s==0.] = 1.
        w_skipgram /= s.reshape((s.shape[0], 1))
        
        index2word_skipgram=model_skipgram.wv.index2word
        word2index_skipgram={key: model_skipgram.wv.vocab[key].index for key in model_skipgram.wv.vocab}
        w_target_skipgram=None
        word2index_target_skipgram=None
        index2word_target_skipgram=None
        
        print ('filter words for context....')
        model_skipgram=rm_stopw_context(model_skipgram)
        
    elif model_type=='a la carte':
        model_w2v=read_w2v(model_param_file)
        w=xp.array(deepcopy(model_w2v.wv.vectors))
        #vector normalize for target w embedding, consistent with context2vec w and convenient for cosine computation among substitutes
        s = xp.sqrt((w * w).sum(1))
        s[s==0.] = 1.
        w /= s.reshape((s.shape[0], 1))
        
        index2word=model_w2v.wv.index2word
        word2index={key: model_w2v.wv.vocab[key].index for key in model_w2v.wv.vocab}
        w_target=None
        word2index_target=None
        index2word_target=None
        print ('filter words for context....')
        model={word:model_w2v.wv.__getitem__(word) for word in model_w2v.wv.vocab}
    
    
    if matrix_f!=None:
        transform=load_transform(matrix_f,{word:model_w2v.wv.__getitem__(word) for word in model_w2v.wv.vocab})
    else:
        transform=None
#     remove stop words in target word space and asarray for computing CI
    print ('filter words for target....')
    w,word2index,index2word=filter_w(w,word2index,index2word)
    if  index2word_target!=None:
        w_target,word2index_target,index2word_target=filter_w(w_target,word2index_target,index2word_target)
    if model_type=='context2vec-skipgram?skipgram':
        w_skipgram,word2index_skipgram,index2word_skipgram=filter_w(w_skipgram,word2index_skipgram,index2word_skipgram)
    
    #### 4. per word weight
    
    w2salience=None
    for wt in weights:
        if wt==LDA:
            print ('load vectors and entropy')
            w2salience=pickle.load(open(w2salience_f))
        elif wt in [INVERSE_W_FREQ, INVERSE_S_FREQ]:
            print ('load w2freq')
            w2salience=load_w2salience(model_w2v,w2salience_f,wt)
    


    ##### 5. combine parameters for skipgram?context2vec-skipgram
    if model_type=='context2vec-skipgram?skipgram':
        model=(model,model_skipgram)
        w=(w,w_skipgram)
        index2word=(index2word,index2word_skipgram)
        word2index=(word2index,word2index_skipgram)
        w2salience=(w2salience,w2salience)
        w_target=(w_target,w_target_skipgram)
        word2index_target=(word2index_target,word2index_target_skipgram)
        index2word_target=(index2word_target,index2word_target_skipgram)
    
    print (model_param_file,model_type,weights,data,w2salience_f)

read model....
Reading config file: ../models/context2vec/model_dir/MODEL-WWCsub-400dim.params.8
Config:  {'config_path': '../models/context2vec/model_dir/', 'model_file': 'MODEL-WWCsub-400dim.8', 'deep': 'yes', 'drop_ratio': '0.0', 'words_file': 'WORDS-WWCsu-400dimb.targets.8', 'unit': '400'}


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:132: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:133: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:134: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


filter words for target....
('../models/context2vec/model_dir/MODEL-WWCsub-400dim.params.8?./eval_data/CRW/vectors.txt', 'context2vec-skipgram', [False], './eval_data/CRW/CRW-562.txt', None)


In [ ]:
    ##### 6. read in data and perform evaluation
    import time
    start_time = time.time()
    print (os.path.basename(os.path.split(data)[0]))
    trials=1
#     data='./eval_data/data-chimeras/dataset_alacarte.l6.fixed.test.txt.punct'
#     data='./eval_data/data-nonces/n2v.definitional.dataset.test.txt'


    if os.path.basename(os.path.split(data)[0])== 'data-chimeras':

            eval_chimera(data,model,model_type,n_result,w,index2word,word2index,weights,w2salience,w_target,word2index_target,index2word_target,M=transform)

    elif os.path.basename(os.path.split(data)[0])== 'data-nonces':
        if context_flag==True:
            eval_nonce_withcontexts(data,model,model_w2v,model_type,n_result,w,index2word,word2index,weights,w2salience,w_target,word2index_target,index2word_target,trials,M=transform)
        else:
            ranks=eval_nonce(data,model,model_w2v,model_type,n_result,w,index2word,word2index,weights,w2salience,w_target,word2index_target,index2word_target,M=transform)

    elif os.path.basename(os.path.split(data)[0])=='CRW':
        
            model_predicts,sps=eval_crw_stf(data,model_param_file,model,model_type,n_result,w,index2word,word2index,weights,w2salience,w_target,word2index_target,index2word_target,trials,M=transform)
    
    elif os.path.basename(os.path.split(data)[0])=='card-660':
#             import pdb; pdb.set_trace()


            eval_card(data,model_param_file,model,model_type,n_result,w,index2word,word2index,weights,w2salience,w_target,word2index_target,index2word_target,trials,contexts=context_flag)
    print("--- %s seconds ---" % (time.time() - start_time))
    
    

CRW

==========processing rareword implication
loading contexts..
0 emptiness: 0.540641
situation: 0.53958505
spirit: 0.53400856
environment: 0.53132
emotions: 0.5303953
truth: 0.53037
circumstances: 0.53016335
sound: 0.5299511
music: 0.5291077
mood: 0.52866495
sweetness: 0.5277291
ambience: 0.52619004
emotion: 0.5257213
lyrics: 0.52536654
smells: 0.5252009
facts: 0.5247296
smell: 0.5246389
landscape: 0.5245816
weather: 0.52457523
taste: 0.524259
1 upshot: 0.53739107
rationale: 0.5326254
difference: 0.5273654
assumption: 0.5253093
outcome: 0.5242594
first: 0.5236402
consensus: 0.5234071
caveat: 0.5232377
hypothesis: 0.5226438
exception: 0.52157456
presumption: 0.5209128
contention: 0.520501
second: 0.5203437
consequence: 0.5202468
theory: 0.5198593
paradox: 0.5194118
conclusion: 0.5192346
result: 0.5189234
problem: 0.51879746
reason: 0.51870435
2 fact: 0.5464128
assumption: 0.5459389
possibility: 0.53968704
premise: 0.5375455
notion: 0.5374823
belief: 0.53645754
supposition: 0.53461254

20 wording: 0.5479774
outcome: 0.5458595
conclusion: 0.54387987
preamble: 0.54126143
verdict: 0.5392592
unconstitutionality: 0.5390053
omission: 0.53832126
text: 0.5367577
publication: 0.5366983
timeliness: 0.5364476
scope: 0.5362918
vagueness: 0.535348
legality: 0.5325659
report: 0.5308418
topicality: 0.530439
rationale: 0.5302204
adjudication: 0.5300277
illegality: 0.52955276
announcement: 0.5293252
validity: 0.5277266
21 motivation: 0.54917705
particularity: 0.543426
consequence: 0.5377515
significance: 0.5366214
rationale: 0.53601795
upshot: 0.53575563
basis: 0.533271
premise: 0.53245395
crux: 0.529466
corollary: 0.5288782
shortcoming: 0.5287458
postulate: 0.5287151
purpose: 0.52807987
aim: 0.5277779
explanation: 0.52725416
plausibility: 0.52695936
peculiarity: 0.5267566
justification: 0.5266317
paradox: 0.52504647
possibility: 0.5246495
22 rationale: 0.55020195
rumour: 0.55014914
reason: 0.54620624
fact: 0.5410751
irony: 0.53957546
supposition: 0.5385296
assumption: 0.537561
alleg

40 crux: 0.54007244
topic: 0.53868645
theme: 0.53529316
highlight: 0.53381455
shortness: 0.5334001
brevity: 0.5317798
particularity: 0.5313989
significance: 0.53112084
premise: 0.53102684
peculiarity: 0.531005
focus: 0.5303244
upshot: 0.5302964
antithesis: 0.5300886
emphasis: 0.5298333
irony: 0.5296318
outcome: 0.52950156
content: 0.5291927
purpose: 0.52911824
oddity: 0.5284493
gist: 0.52790886
41 assumption: 0.5489463
notion: 0.54791915
fact: 0.5454797
idea: 0.5452369
mindset: 0.5426451
premise: 0.54251415
supposition: 0.542256
presumption: 0.5403368
view: 0.53945
theory: 0.5393059
expectation: 0.53740877
realization: 0.5369367
belief: 0.5367049
presupposition: 0.5367031
assumptions: 0.53535634
perception: 0.5345673
misperception: 0.53332555
realisation: 0.53238654
rationale: 0.5322767
assertion: 0.53156817
42 indication: 0.546175
supposition: 0.5453412
doubt: 0.53345156
presumption: 0.53304785
meaning: 0.5323476
certainty: 0.53037095
pretence: 0.5299061
surmise: 0.52814204
reason: 0.

60 upshot: 0.54524606
result: 0.5364664
consequence: 0.53579056
reward: 0.534681
supposition: 0.53369874
rationale: 0.53352386
difference: 0.5319914
watchword: 0.53091884
outcome: 0.53066194
pretext: 0.52999485
consensus: 0.5282903
exception: 0.5280886
intention: 0.5280489
assumption: 0.52761877
verdict: 0.5271195
reason: 0.5261585
presumption: 0.52531666
concession: 0.52489156
conclusion: 0.52488595
motivation: 0.52437204
61 notation: 0.5419292
libraries: 0.54192704
delimiters: 0.5415631
diagrams: 0.54138905
codes: 0.5388595
sequences: 0.537863
notations: 0.5375908
identifiers: 0.5374925
dictionaries: 0.53623784
schemes: 0.53616726
lists: 0.53547215
rules: 0.53546995
symbols: 0.5354203
statements: 0.53426164
tables: 0.5340928
syntax: 0.53239745
patterns: 0.53239644
schemas: 0.53215295
graphs: 0.53171647
prefixes: 0.53126246
62 drawback: 0.54458046
feature: 0.5379734
use: 0.532847
innovation: 0.5311688
problem: 0.52891314
differentiator: 0.5286951
application: 0.5276831
creation: 0.526

80 ordinator: 0.5652907
ordinating: 0.54318935
operation: 0.5398819
ordination: 0.5391667
conceptualization: 0.539007
internalization: 0.53867155
occurrence: 0.53864735
fashioning: 0.53833556
imagining: 0.5370804
objectification: 0.5353385
activation: 0.53513765
expression: 0.5345922
articulation: 0.53455544
creation: 0.534276
perceiving: 0.53319347
apprehension: 0.53258246
object: 0.53242105
actualization: 0.5319417
interpretation: 0.53156316
realisation: 0.5313225
81 possibility: 0.55422705
presumption: 0.55325025
supposition: 0.55099803
belief: 0.544968
notion: 0.54418874
expectation: 0.5441536
assumption: 0.5434731
indication: 0.54319483
misperception: 0.54265577
opinion: 0.5420401
contention: 0.5411099
perception: 0.5400589
assertion: 0.53966606
argument: 0.53946865
insinuation: 0.53946614
premise: 0.5391149
dictum: 0.53838295
idea: 0.53793705
stipulation: 0.53791046
concern: 0.5374483
82 realizing: 0.53186905
discovering: 0.5258602
rumours: 0.5251218
rumors: 0.52432644
deducing: 

...

145 distortions: 0.5392471
extrapolation: 0.5388767
analysis: 0.53521436
estimates: 0.5346499
correlations: 0.53445697
substantiation: 0.53271645
calculation: 0.5309287
measurements: 0.5292652
comparisons: 0.52853
assessment: 0.5284441
estimation: 0.5280972
measurement: 0.52803475
indices: 0.52651316
estimations: 0.525612
analyses: 0.52551717
simplifications: 0.52504116
calculations: 0.523979
interpretations: 0.5239787
quantification: 0.5239325
predictions: 0.5225313
146 exemptions: 0.5477931
receipts: 0.5432764
payers: 0.5392061
tolls: 0.53901744
rates: 0.53725064
moneys: 0.53674406
quotas: 0.5359065
reliefs: 0.5356558
deductions: 0.5337059
allocations: 0.53193146
burdens: 0.5318451
allowances: 0.5318013
dues: 0.53102714
restrictions: 0.530498
barriers: 0.5304401
allotments: 0.53
indemnities: 0.52995664
rebates: 0.52967
levies: 0.52946264
credits: 0.5292003
147 response: 0.54220396
controversy: 0.5419902
reception: 0.5392652
reaction: 0.537084
reactions: 0.5370318
praise: 0.53696734
c

164 criticism: 0.54696184
critique: 0.5439208
condemnation: 0.54371744
misinterpretations: 0.5380461
violations: 0.537641
denigration: 0.5374971
rejection: 0.5362108
appraisals: 0.536154
denial: 0.5360527
acknowledgment: 0.53500086
interpretations: 0.53416175
recognition: 0.53414613
skepticism: 0.5332216
repudiation: 0.53316635
explanations: 0.53297246
suppression: 0.5328748
assertions: 0.53205514
critiques: 0.5320389
distortions: 0.53096116
misconceptions: 0.5307436
165 strategies: 0.5582533
considerations: 0.5495856
objectives: 0.5464715
measures: 0.546353
initiatives: 0.54568017
factors: 0.5396512
decisions: 0.5396467
enhancements: 0.5385743
actions: 0.53727823
choices: 0.53688776
challenges: 0.53677666
outcomes: 0.53522277
deadlines: 0.5340278
scenarios: 0.5327906
procedures: 0.5315883
interventions: 0.53156644
evaluations: 0.53115654
opportunities: 0.53040767
strategy: 0.53007674
policies: 0.5297452
166 sources: 0.558413
reports: 0.5519492
researchers: 0.5485554
observers: 0.54709

183 clinics: 0.54372793
services: 0.5422798
procedures: 0.5415474
hotlines: 0.5317228
registries: 0.53106594
databases: 0.5309294
descriptors: 0.5296341
treatments: 0.5282116
therapies: 0.5271656
medications: 0.5268278
programs: 0.5264226
classifications: 0.52618766
facilities: 0.5255983
practices: 0.52427197
diagnoses: 0.52348256
provisions: 0.52271897
protections: 0.521642
heuristics: 0.52080804
aftercare: 0.51998615
evaluations: 0.51994896
184 associates: 0.55657226
co.: 0.5560655
partners: 0.55472136
consulting: 0.548172
filmworks: 0.5344314
leventhal: 0.5314866
solutions: 0.5308401
diagnostics: 0.5297109
company: 0.5296006
ors: 0.52926505
dorr: 0.5289825
mbh: 0.5266565
int'l: 0.5264104
research: 0.5263873
stepper: 0.5256679
equity: 0.5255448
sohn: 0.52531767
amp: 0.52500343
acquisitions: 0.5246681
hankins: 0.52400905
185 analysis: 0.57735914
analyses: 0.5578047
evaluation: 0.550715
comparisons: 0.5506457
computations: 0.5502372
verification: 0.546048
calculations: 0.5433993
indice

203 commendations: 0.54392946
praise: 0.54254156
interpretations: 0.54228556
eulogies: 0.54087883
opinions: 0.54019296
plaudits: 0.5399125
views: 0.53854376
recognition: 0.53829587
testimonials: 0.5375708
condemnations: 0.5360151
accolades: 0.53534085
criticism: 0.5337831
reviews: 0.5323383
praises: 0.53089994
scorn: 0.5304617
readings: 0.5287514
reception: 0.5280848
reputations: 0.52786094
acclaim: 0.5267802
approbation: 0.526131
204 treatise: 0.5278069
examples: 0.5275922
stradivarius: 0.5275634
cradle: 0.52635616
imitators: 0.52578217
imitations: 0.52545786
history: 0.52329206
forerunners: 0.52313143
master: 0.52294296
critique: 0.52272546
birthplace: 0.52261907
disciples: 0.52260166
symphonies: 0.5225116
genius: 0.5222429
caricatures: 0.5218823
violins: 0.521363
contemporaries: 0.5209801
stoning: 0.5209314
criticism: 0.5205092
acquaintances: 0.52014697
205 courses: 0.5622774
instruction: 0.5478412
coursework: 0.5472222
education: 0.54213
schooling: 0.5396908
examinations: 0.5376684

223 disparagement: 0.5443389
earnestness: 0.54283386
nature: 0.54046434
reevaluation: 0.5386053
appreciation: 0.53825927
appraisal: 0.5316934
excellence: 0.53091633
quality: 0.5293513
poignancy: 0.52894276
glorification: 0.52870363
merits: 0.5281773
virtuosity: 0.52767885
frankness: 0.5276643
appraisals: 0.52756804
ambience: 0.5271861
significance: 0.5271859
immediacy: 0.5267899
realism: 0.52656645
tenor: 0.5263184
reappraisal: 0.5258309
224 archaeology: 0.53998697
literature: 0.53868824
findings: 0.53770596
evidence: 0.53647494
contexts: 0.5361428
documentation: 0.53493017
analyses: 0.53465146
digs: 0.53435224
studies: 0.53381217
investigations: 0.5327158
analysis: 0.5301134
surveys: 0.5297736
documents: 0.5284982
research: 0.52848995
records: 0.52574944
reconstructions: 0.5250932
texts: 0.5243465
artifacts: 0.52321196
evidences: 0.52227575
stratigraphy: 0.5220678
225 challenges: 0.54401654
tasks: 0.5432283
objectives: 0.5423924
activities: 0.5402449
strategies: 0.5389922
learning: 0.